# Regression with scikit-learn using Soccer Dataset

We are using the open dataset from Kaggle that we used in Week 1 for our example

Recall that this European Soccer Database has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016

## Import Libraries

In [42]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

## Read Data from the Database into pandas

In [3]:
# Create the connection to database sqlite
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM PLayer_Attributes", cnx)

In [13]:
print(df.shape)
df.head()

(183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [14]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master 
WHERE type='table';"""

# Creating cursor object using connection object
cursor = cnx.cursor()

# executing our sql query
cursor.execute(sql_query)

# printing all tables list
print("List of tables\n")
print(cursor.fetchall())

List of tables

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


## Declare the columns you want to use as Features

In [15]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [16]:
features = ['potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

## Specify the Prediction Target

In [17]:
target = ['overall_rating']

## Clean the Data
We already know this data has many NaN values so we will drop them

In [18]:
df = df.dropna()

## Extract Features and Target ('overall_rating") Values into Separate Dataframes

In [22]:
X = df[features]
X.shape

(180354, 34)

In [23]:
y = df[target]
y.shape

(180354, 1)

Let's look at a typical row from our features:

In [21]:
X.iloc[2]

potential             66.0
crossing              49.0
finishing             44.0
heading_accuracy      71.0
short_passing         61.0
volleys               44.0
dribbling             51.0
curve                 45.0
free_kick_accuracy    39.0
long_passing          64.0
ball_control          49.0
acceleration          60.0
sprint_speed          64.0
agility               59.0
reactions             47.0
balance               65.0
shot_power            55.0
jumping               58.0
stamina               54.0
strength              76.0
long_shots            35.0
aggression            63.0
interceptions         41.0
positioning           45.0
vision                54.0
penalties             48.0
marking               65.0
standing_tackle       66.0
sliding_tackle        69.0
gk_diving              6.0
gk_handling           11.0
gk_kicking            10.0
gk_positioning         8.0
gk_reflexes            8.0
Name: 2, dtype: float64

Let's look at the target (y) values

In [26]:
print(y.min(), y.max())

overall_rating    33.0
dtype: float64 overall_rating    94.0
dtype: float64


In [29]:
# find the stats for target value
y.describe()

,overall_rating
count,180354.000000
mean,68.635317
std,7.027950
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


## Split the dataset into Training and Test Datasets

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

## (1) Linear Regression: Fit a model to the training set

In [31]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

### Perform Prediction using Linear Regression Model

In [48]:
y_prediction = regressor.predict(X_test)
print(y_prediction.shape)
print(y_prediction.dtype)
y_prediction

(59517, 1)
float64


array([[66.51284879],
       [79.77234615],
       [66.57371825],
       ...,
       [69.23780133],
       [64.58351696],
       [73.6881185 ]])

### What is the mean of expected target value in the test set?

In [54]:
print(y_test.shape)
y_test.describe()

(59517, 1)


,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


In [69]:
#convert to pandas Dataframe object
y_prediction = pd.DataFrame(y_prediction)
LinearReg_Stats = y_prediction.describe()
LinearReg_Stats

,0
count,59517.000000
mean,68.629307
std,7.013511
min,33.000000
25%,64.000000
50%,68.944444
75%,73.205128
max,94.000000


### Evaluate Linear Regression Accuracy using Root Mean Square Error

In [59]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
RMSE

2.805303046855224

## (2) Decision Tree Regressor: Fit  new regression model to the training set

In [60]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### Perform Prediction using Decision Tree Regressor

In [61]:
y_prediction = regressor.predict(X_test)
y_prediction

array([62.        , 84.        , 62.38666667, ..., 71.        ,
       62.        , 73.        ])

### For comparision: What is the mean of the expected target value in the test set?

In [62]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


### Evaluate Decision Tree Regression Accuracy using Root Mean Square Error

In [63]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
RMSE

1.4514246429979463

In [73]:
y_prediction = pd.DataFrame(y_prediction)
DecisionTree_Stats = y_prediction.describe()
DecisionTree_Stats

,0
count,59517.000000
mean,68.629307
std,7.013511
min,33.000000
25%,64.000000
50%,68.944444
75%,73.205128
max,94.000000


In [72]:
Results = pd.concat([LinearReg_Stats, DecisionTree_Stats], axis=1)
Results

,0,0
count,59517.000000,59517.000000
mean,68.629307,68.629307
std,7.013511,7.013511
min,33.000000,33.000000
25%,64.000000,64.000000
50%,68.944444,68.944444
75%,73.205128,73.205128
max,94.000000,94.000000


### Results Comparison:

RMSE for Linear Regression = 2.805303046855224

RMSE for DecisionTree = 1.4514246429979463

Therefore Decision Tree gave better result since error is smaller!